In [1]:
# !pip install pdfquery
# !pip install pandas
# !pip install PyPDF2
# !pip install --upgrade pymupdf
# !pip install requests

In [1]:
# import pdfquery
import pandas as pd
import os
import fitz
import requests
import csv
# import PyPDF2

In [2]:
os.chdir(r"C:\Users\SeanS\Documents\CS 6422\EvaDB PDF\Sample1")

In [3]:
# Was used to just get first page
# import fitz  # ensure you've imported the necessary library
# doc = fitz.open("spring_2023_orientation_document.pdf") 
# out = open("spring_2023_orientation_document_p1_output.txt", "wb") 
# page = doc[1]
# text = page.get_text().encode("utf8")
# out.write(text) 
# out.close()

# For Entire Document
doc = fitz.open("spring_2023_orientation_document.pdf") # open a document
out = open("spring_2023_orientation_document_output.txt", "wb") # create a text output
for page in doc: # iterate the document pages
    text = page.get_text().encode("utf8") # get plain text (is in UTF-8)
    out.write(text) 
    out.write(bytes((12,))) # write page delimiter (form feed 0x0C)
out.close()

In [4]:
with open("spring_2023_orientation_document_output.txt", "r", encoding='utf-8') as f:
    lines = f.readlines()

sentences = []
current_sentence = ''

for line in lines:
    stripped_line = line.strip()
    if not stripped_line:
        # skip empty lines
        continue  
    # If the line looks like a title (no ending punctuation and possibly short)
    if (len(stripped_line) < 60 and not stripped_line.endswith(('.', '!', '?'))):
        if current_sentence:
            sentences.append(current_sentence)
            current_sentence = ''
        # Treat title as its own 'sentence'
        sentences.append(stripped_line)  
        continue
    # If the line ends with a typical sentence-ending punctuation
    if stripped_line.endswith(('.', '!', '?')):
        current_sentence += ' ' + stripped_line
        sentences.append(current_sentence)
        current_sentence = ''
    else:
        current_sentence += ' ' + stripped_line

# For the case where the last piece of text doesn't end with a sentence-ending punctuation
if current_sentence:
    sentences.append(current_sentence)
    
with open('combined_sentences.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
        f.write(sentence + '\n')

In [16]:
url = 'https://api.unstructured.io/general/v0/general'

headers = {
    'accept': 'application/json',
    'unstructured-api-key': 'DTmC4RIyD6E3XMBJcucb3GOUMoKHWK',
}

data = {
    "strategy": "ocr_only",
    "ocr_languages": ["eng", "kor"]
}

file_path = "combined_sentences.txt"
file_data = {'files': open(file_path, 'rb')}

response = requests.post(url, headers=headers, data=data, files=file_data)

file_data['files'].close()

structured_data = response.json()

In [17]:
df = pd.DataFrame(structured_data)
print(df)

               type                        element_id  \
0             Title  71ff43c910647e81d39e44c90f1cda0b   
1             Title  492adf374353f048a3fcae9a48578d49   
2             Title  3bcd9e03e3eaa295031a24cf9838cbde   
3     NarrativeText  705eac3b0c92b0b88ed6179a97b0a8c7   
4     NarrativeText  1f1bc56e00d904291eb1716c7a6782fc   
...             ...                               ...   
1417          Title  5fea0f0a7249db6476df8c65e4d17696   
1418          Title  661618e17731986974fb3da731b1b96b   
1419          Title  c3d24dae34dc01c8baa600dd8d981014   
1420   EmailAddress  03861bef551800ced62de49b5e9b8a02   
1421  NarrativeText  fa47d600d89f84525001de8f0e5bbaf5   

                                               metadata  \
0     {'filename': 'combined_sentences.txt', 'filety...   
1     {'filename': 'combined_sentences.txt', 'filety...   
2     {'filename': 'combined_sentences.txt', 'filety...   
3     {'filename': 'combined_sentences.txt', 'filety...   
4     {'filename': '

In [19]:
df2 = pd.json_normalize(structured_data)
print(df2)

               type                        element_id  \
0             Title  71ff43c910647e81d39e44c90f1cda0b   
1             Title  492adf374353f048a3fcae9a48578d49   
2             Title  3bcd9e03e3eaa295031a24cf9838cbde   
3     NarrativeText  705eac3b0c92b0b88ed6179a97b0a8c7   
4     NarrativeText  1f1bc56e00d904291eb1716c7a6782fc   
...             ...                               ...   
1417          Title  5fea0f0a7249db6476df8c65e4d17696   
1418          Title  661618e17731986974fb3da731b1b96b   
1419          Title  c3d24dae34dc01c8baa600dd8d981014   
1420   EmailAddress  03861bef551800ced62de49b5e9b8a02   
1421  NarrativeText  fa47d600d89f84525001de8f0e5bbaf5   

                                                   text  \
0     Online Master of Science in Computer Science (...   
1                                           Spring 2023   
2                Spring 2023 OMSCS Orientation Document   
3     Welcome to the OMSCS program! The faculty and ...   
4     We encourage y

In [20]:
df.to_csv("output_table_combined.txt", index=False)

Ignore Below, which was for exploration

In [ ]:
# print(structured_data)
df = pd.read_json(response.json)

In [ ]:
# Assuming structured_data is the parsed JSON you got from unstructured.io
sections = structured_data['sections']

# Define the CSV file path
csv_path = "unstructuredio.csv"

# Write the structured data to the CSV
with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the header
    writer.writerow(["Title", "Content"])
    
    # Write the sections
    for section in sections:
        writer.writerow([section['title'], section['content']])

In [ ]:
# Define the TXT file path
txt_path = "unstructuredio.txt"

# Write the structured data to the TXT file
with open(txt_path, 'w', encoding='utf-8') as txtfile:
    for section in sections:
        txtfile.write("Title: " + section['title'] + '\n')
        txtfile.write("Content:\n" + section['content'] + '\n\n')

In [ ]:
doc = fitz.open("spring_2023_orientation_document.pdf") # open a document
out = open("spring_2023_orientation_document_output.txt", "wb") # create a text output
for page in doc: # iterate the document pages
    text = page.get_text().encode("utf8") # get plain text (is in UTF-8)
    out.write(text) # write text of page
    out.write(bytes((12,))) # write page delimiter (form feed 0x0C)
out.close()

In [ ]:
for page_index in range(len(doc)): # iterate over pdf pages
    page = doc[page_index] # get the page
    image_list = page.get_images()

    # print the number of images found on the page
    if image_list:
        print(f"Found {len(image_list)} images on page {page_index}")
    else:
        print("No images found on page", page_index)

    for image_index, img in enumerate(image_list, start=1): # enumerate the image list
        xref = img[0] # get the XREF of the image
        pix = fitz.Pixmap(doc, xref) # create a Pixmap

        if pix.n - pix.alpha > 3: # CMYK: convert to RGB first
            pix = fitz.Pixmap(fitz.csRGB, pix)

        pix.save("page_%s-image_%s.png" % (page_index, image_index)) # save the image as png
        pix = None
doc.close()

In [ ]:
os.chdir(r"C:\Users\SeanS\Documents\CS 6422\EvaDB PDF\Sample2")

In [ ]:
doc2 = fitz.open("pdf_sample1.pdf") # open a document
out = open("pdf_sample1.txt", "wb") # create a text output
for page in doc2: # iterate the document pages
    text = page.get_text().encode("utf8") # get plain text (is in UTF-8)
    out.write(text) # write text of page
    out.write(bytes((12,))) # write page delimiter (form feed 0x0C)
out.close()

In [ ]:
for page_index in range(len(doc2)): # iterate over pdf pages
    page = doc2[page_index] # get the page
    image_list = page.get_images()

    # print the number of images found on the page
    if image_list:
        print(f"Found {len(image_list)} images on page {page_index}")
    else:
        print("No images found on page", page_index)

    for image_index, img in enumerate(image_list, start=1): # enumerate the image list
        xref = img[0] # get the XREF of the image
        pix = fitz.Pixmap(doc2, xref) # create a Pixmap

        if pix.n - pix.alpha > 3: # CMYK: convert to RGB first
            pix = fitz.Pixmap(fitz.csRGB, pix)

        pix.save("page_%s-image_%s.png" % (page_index, image_index)) # save the image as png
        pix = None
doc2.close()

In [ ]:
# Testing with pdfquery
# pdf = pdfquery.PDFQuery('spring_2023_orientation_document.pdf')
# pdf.load()
# pdf.tree.write('test.xml', pretty_print = True)
# pdf

In [ ]:
# Testing with PyPDF2
# Open in binary mode
# pdf_file = open('spring_2023_orientation_document.pdf', 'rb')
# pdf_reader = PyPDF2.PdfReader(pdf_file)

In [ ]:
# Testing with PyPDF2
page = pdf_reader.pages[1]
page_text = page.extract_text()
print(page_text)